### Library Imports

In [ ]:
import numpy as np
import random
import cv2
from imutils import paths
import os

# SkLearn Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

# TensorFlow Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K


### Data Loading

In [ ]:
def load_mnist_data(dataroot):

    X = list()
    y = list()

    for label in os.listdir(dataroot):
      label_dir_path = dataroot + "/"+label

      for imgFile in os.listdir(label_dir_path):
        img_file_path = label_dir_path + "/" + imgFile
        image_gray = cv2.imread(img_file_path, cv2.IMREAD_GRAYSCALE)

        image = np.array(image_gray).flatten()

        X.append(image/255)
        y.append(label)

    return X, y


### Client Node Creation

In [ ]:
def create_client_nodes(X, 
                        y, 
                        num_clients=10, 
                        prefix='CLIENT_'):

    #create a list of client names
    client_names = []
    for i in range(num_clients):
      client_names.append(prefix + str(i))

    #randomize the data
    data = list(zip(X, y))
    random.shuffle(data)

    #shard data and place at each client
    per_client = len(data)//num_clients
    client_chunks = []
    start = 0
    end = 0

    for i in range(num_clients):
      end = start + per_client
      if end > len(data):
        client_chunks.append(data[start:])
      else:
        client_chunks.append(data[start:end])
        start = end 

    return {client_names[i] : client_chunks[i] for i in range(len(client_names))} 


In [ ]:
def collapse_chunk(chunk, batch_size=32):

    X, y = zip(*chunk)
    dataset = tf.data.Dataset.from_tensor_slices((list(X), list(y)))
    return dataset.shuffle(len(y)).batch(batch_size)


### Classification Model

In [ ]:
def MNIST_DeepLearning_Model(hidden_layer_sizes = [200, 200, 200]):
  input_dim = 784
  num_classes = 10

  model = Sequential()

  model.add(Dense(200, input_shape=(input_dim,)))
  model.add(Activation("relu"))

  for hidden in hidden_layer_sizes:
    model.add(Dense(hidden))
    model.add(Activation("relu"))

  model.add(Dense(num_classes))
  model.add(Activation("softmax"))
  
  return model


### Weight Scaling

In [ ]:
def scale_weights(all_clients,
                  this_client,
                  weights):
  
  # First calculate scaling factor

  # Obtain batch size
  batch_size = list(all_clients[this_client])[0][0].shape[0]

  # Compute global data size
  sizes = []
  for client in all_clients.keys():
    sizes.append(tf.data.experimental.cardinality(all_clients[client]).numpy())
  global_data_size = np.sum(sizes)*batch_size

  # Compute data size in this client 
  this_client_size = tf.data.experimental.cardinality(all_clients[this_client]).numpy()*batch_size

  # Scaling factor is the ratio of the two 
  scaling_factor = this_client_size / global_data_size

  scaled_weights = []
  for weight in weights:
    scaled_weights.append(scaling_factor * weight)

  return scaled_weights


### Global Model

In [ ]:
global_model = MNIST_DeepLearning_Model(hidden_layer_sizes = [200, 200, 200])
global_model.summary()


In [ ]:
dataroot = './trainingSet'
X, y = load_mnist_data(dataroot)

y_binarized = LabelBinarizer().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                      y_binarized, 
test_size=0.2, 
random_state=123)


### Client Data Distribtion

In [ ]:
clients = create_client_nodes(X_train, y_train, num_clients=10)
clients_data = {}
for client_name in clients.keys():
    clients_data[client_name] = collapse_chunk(clients[client_name])
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))


In [ ]:
learn_rate = 0.01 
num_rounds = 40
loss='categorical_crossentropy'
metrics = ['accuracy']


### Federated Learning

In [ ]:
for round in range(num_rounds):
            
    # Get the weights of the global model
    global_weights = global_model.get_weights()
    

    scaled_local_weights = []

    # Shuffle the clients
    # This will remove any inherent bias
    client_names= list(clients_data.keys())
    random.shuffle(client_names)
    
    # Create initial local models 
    for client in client_names:

        # Create the model
        local_client_model = MNIST_DeepLearning_Model(hidden_layer_sizes = [200])

        # Compile the model
        local_client_model.compile(loss=loss, 
                                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                                   metrics=metrics)
        
        # The model will have random weights
        # We need to reset it to the weights of the current global model
        local_client_model.set_weights(global_weights)
        
        # Train local model 
        local_client_model.fit(clients_data[client], 
                               epochs=1,
                               verbose = 0)
        
        # Scale model weights 
        # Based on this client model's local weights
        scaled_weights = scale_weights(clients_data, client, local_client_model.get_weights())

        # Record the value
        scaled_local_weights.append(scaled_weights)
        
        # Memory management
        K.clear_session()
        
    # Communication round has ended
    # Need to compute the average gradients from all local models 
    average_weights = []
    for gradients in zip(*scaled_local_weights):
        # Calculate mean per-layer weights
        layer_mean = tf.math.reduce_sum(gradients, axis=0)

        # This becomes new weight for that layer
        average_weights.append(layer_mean)

    # Update global model with newly computed gradients
    global_model.set_weights(average_weights)

    # Evaluate performance of model at end of round
    losses = []
    accuracies = []
    for(X_test, Y_test) in test_batched:
        # Use model for inference
        Y_pred = global_model.predict(X_test)

        # Calculate loss based on actual and predicted value
        loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        loss_value = loss_fn(Y_test, Y_pred)
        losses.append(loss_value)

        # Calculate accuracy based on actual and predicted value
        accuracy_value = accuracy_score(tf.argmax(Y_pred, axis=1), 
                                       tf.argmax(Y_test, axis=1))
        accuracies.append(accuracy_value)

    # Print Information
    print("ROUND: {} ---------- GLOBAL ACCURACY: {:.2%}".format(round, accuracy_value))


### Visualizing Loss and Accracy Trends

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(range(num_rounds), losses)
plt.xlabel("Communication Rounds")
plt.ylabel("Loss")


In [ ]:
import matplotlib.pyplot as plt 
plt.plot(range(num_rounds), accuracies)
plt.xlabel("Communication Rounds")
plt.ylabel("Accuracy")


### Privacy Utility Tradeoff: No Privacy

In [ ]:
# Initialize global model
global_model = MNIST_DeepLearning_Model(hidden_layer_sizes = [200, 200, 200])
global_model.compile(loss=loss, 
                     optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                     metrics=metrics)

# Create dataset from entire data
full_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))\
                              .shuffle(len(y_train))\
                              .batch(32)

# Fit the model
global_model.fit(full_dataset, epochs = 10)


### Privacy Utility Tradeoff: Full Privacy

In [ ]:
# Initialize local client 
local_client_model = MNIST_DeepLearning_Model(hidden_layer_sizes = [200, 200, 200])
local_client_model.compile(loss=loss, 
                     optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                     metrics=metrics)

# Train on only one client data
local_client_model.fit(clients_data['CLIENT_8'], 
                               epochs=10)
